In [129]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [130]:
D=pd.read_excel("./카테고리_소분류완료.xlsx")
data=pd.DataFrame(D)
data.drop(columns='Unnamed: 0',inplace=True)
plt.rcParams["font.family"]="BM YEONSuNG"

In [131]:
범주형=['(가)건강 만족도', '(나)가족의 수입 만족도', '(다)주거 환경 만족도', '(라)가족관계 만족도', '(마)직업 만족도', '(바)사회적 친분관계 만족도', '(사)여가생활 만족도', '성별','교육수준', '종교','만성질환', '건강상태', '주택유형', '집의 (등기상) 점유형태', '근로유형', '혼인상태']
연속형=["총생활비","연령","주거면적","방수"]

# 범주형과 연속형 사이의 독립성 검정

## 범주형과 연속형 사이의 combination

In [132]:
import itertools
comb=[list(x) for x in itertools.product(범주형,연속형)]

### 등분산성
F-test 진행

In [133]:
import scipy as sp

In [134]:
# 그룹
dc={}
for x in data[comb[0][0]].unique():
    temp=pd.DataFrame(data[data[comb[0][0]]==x][comb[0][1]])
    dc[x]=temp.reset_index().drop("index",axis=1)

In [135]:
formula=",".join([f'dc[{i}]' for i in dc.keys()])

In [136]:
list(map(lambda x:x[0],sp.stats.f_oneway(dc[2],dc[5],dc[4],dc[3],dc[1])))

[101.74301392534885, 7.696997783169532e-82]

#### F-test

In [137]:
result=[]
for c in comb:
    part_result,dc={},{}
    for x in data[c[0]].unique():
        temp=pd.DataFrame(data[data[c[0]]==x][c[1]])
        dc[x]=temp.reset_index().drop("index",axis=1)
    formula=",".join([f'dc[{i}]' for i in dc.keys()])
    part_result['combination']=c
    part_result["pvalue"]=eval(f'sp.stats.f_oneway({formula})')[1][0]
    result.append(part_result)

In [138]:
df=pd.DataFrame(result)

In [139]:
df[df["pvalue"]>=0.05]

,combination,pvalue
9,"[(다)주거 환경 만족도, 연령]",0.063552
19,"[(마)직업 만족도, 방수]",0.323229
23,"[(바)사회적 친분관계 만족도, 방수]",0.076107
27,"[(사)여가생활 만족도, 방수]",0.804913


#### 정규성 가정이 없다면?

In [140]:
result2=[]
for c in comb:
    part_result,dc={},{}
    for x in data[c[0]].unique():
        temp=pd.DataFrame(data[data[c[0]]==x][c[1]])
        dc[x]=temp.reset_index().drop("index",axis=1)
    formula=",".join([f'dc[{i}].squeeze()' for i in dc.keys()])
    part_result['combination']=c
    part_result["pvalue"]=eval(f'sp.stats.levene({formula}).pvalue')
    result2.append(part_result)


In [141]:
df=pd.DataFrame(result)
df[df["pvalue"]>=0.05]

,combination,pvalue
9,"[(다)주거 환경 만족도, 연령]",0.063552
19,"[(마)직업 만족도, 방수]",0.323229
23,"[(바)사회적 친분관계 만족도, 방수]",0.076107
27,"[(사)여가생활 만족도, 방수]",0.804913


### anova

In [142]:
import statsmodels.stats.anova as anova
import statsmodels.formula.api as sf
import statsmodels.api as sm

In [143]:
data[['(다)주거 환경 만족도','총생활비']]
add_exog=sm.add_constant(data[['(다)주거 환경 만족도']])

In [144]:
model=sm.OLS(endog=data["총생활비"],exog=add_exog)

In [145]:
fit=model.fit()

In [146]:
fit.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   총생활비   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     15.38
Date:                Wed, 16 Aug 2023   Prob (F-statistic):           8.95e-05
Time:                        15:47:06   Log-Likelihood:                -20817.
No. Observations:                3334   AIC:                         4.164e+04
Df Residuals:                    3332   BIC:                         4.165e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          111.4655     12.319      9.048      0.000      87.311     135.619
(다)주거 환경 만족도    13.3326      3.399      3.922      0.000       6.668      19.997
==============================================================================
Omnibus:                     3966.616   Durbin-Watson:                   1.246
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           885609.350
Skew:                           5.989   Prob(JB):                         0.00
Kurtosis:                      81.941   Cond. No.                         22.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [147]:
list(fit.pvalues)[1]

8.949568651699636e-05

In [148]:
anova_result=[]
for col in [x["combination"] for x in result]:
    temp={}
    model=sm.OLS(endog=data[col[1]],exog=sm.add_constant(data[col[0]])).fit()
    temp["combination"],temp["pvalue"]=col,model.pvalues[1]
    anova_result.append(temp)

In [149]:
df=pd.DataFrame(anova_result)
df[df["pvalue"]>=0.05]

,combination,pvalue
7,"[(나)가족의 수입 만족도, 방수]",0.464404
9,"[(다)주거 환경 만족도, 연령]",0.080135
19,"[(마)직업 만족도, 방수]",0.287758
23,"[(바)사회적 친분관계 만족도, 방수]",0.971917
27,"[(사)여가생활 만족도, 방수]",0.510343


In [150]:
anova_result2=[]
for col in [x["combination"] for x in result2]:
    temp={}
    model=sm.OLS(endog=data[col[1]],exog=sm.add_constant(data[col[0]])).fit()
    temp["combination"],temp["pvalue"]=col,model.pvalues[1]
    anova_result2.append(temp)

In [151]:
df2=pd.DataFrame(anova_result2)
final=df2[df2["pvalue"]>=0.05]

In [153]:
final["combination"].values

array([list(['(나)가족의 수입 만족도', '방수']), list(['(다)주거 환경 만족도', '연령']),
       list(['(마)직업 만족도', '방수']), list(['(바)사회적 친분관계 만족도', '방수']),
       list(['(사)여가생활 만족도', '방수'])], dtype=object)